In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime

In [2]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 04A9-8968

 C:\Users\나\Desktop\업무\Data Analysis 디렉터리

2020-12-17  오후 02:36    <DIR>          .
2020-12-17  오후 02:36    <DIR>          ..
2020-12-17  오후 02:35    <DIR>          .ipynb_checkpoints
2020-09-26  오전 05:40     3,248,970,693 AL_11_D151_20200925.csv
2020-12-11  오전 10:14         7,231,071 coor_data_df.csv
2020-12-07  오후 03:32           882,655 coor_df.csv
2020-11-21  오후 07:52               315 debug.log
2020-12-17  오후 02:36            40,689 EDA 2.ipynb
2020-12-17  오후 02:35                72 EDA 3.ipynb
2020-12-03  오후 03:45           245,179 EDA.ipynb
2020-11-24  오후 03:05        87,232,378 entrc_seoul.txt
2020-12-11  오후 05:10           123,559 Feature Engineering 2.ipynb
2020-12-11  오전 10:20           448,823 Feature Engineering.ipynb
2020-12-04  오후 04:45         5,050,450 get_coor.csv
2020-12-07  오후 12:52               115 juso_api.txt
2020-12-04  오후 05:45                43 kakao_api.txt
2020-12-02  오후 05:00           348,527 Practice.ipynb
20

# Read Data

In [3]:
def file_reader(basedir, filename):
    if filename.endswith('.csv'):
        try:
            df = pd.read_csv(basedir + filename, header=15)
        except:
            df = pd.read_csv(basedir + filename, encoding='euc-kr', header=15)
    elif filename.endswith('.xlsx'):
        try:
            df = pd.read_excel(basedir + filename, header=16)
        except:
            df = pd.read_excel(basedir + filename, encoding='euc-kr', header=16)
    else:
        print('error')
    
    return df

In [4]:
def read_real_estate_prices_df():
    basedir = './국토교통부_실거래가_공개시스템/'
    filenames = os.listdir(basedir)
    
    apart_trade_files = [filename for filename in filenames if filename.startswith('아파트(매매)')]
    apart_lease_files = [filename for filename in filenames if filename.startswith('아파트(전월세)')]
    multiplex_trade_files = [filename for filename in filenames if filename.startswith('연립다세대(매매)')]
    multiplex_lease_files = [filename for filename in filenames if filename.startswith('연립다세대(전월세)')]
    officetel_trade_files = [filename for filename in filenames if filename.startswith('오피스텔(매매)')]
    officetel_lease_files = [filename for filename in filenames if filename.startswith('오피스텔(전월세)')]
    
    #
    apart_trade_dfs_list = []
    for filename in apart_trade_files:
        df = file_reader(basedir, filename)
        apart_trade_dfs_list.append(df)
    apart_trade_df = pd.concat(apart_trade_dfs_list).reset_index(drop=True)
    
    #
    apart_lease_dfs_list = []
    for filename in apart_lease_files:
        df = file_reader(basedir, filename)
        apart_lease_dfs_list.append(df)
    apart_lease_df = pd.concat(apart_lease_dfs_list).reset_index(drop=True)
    
    #
    multiplex_trade_dfs_list = []
    for filename in multiplex_trade_files:
        df = file_reader(basedir, filename)
        multiplex_trade_dfs_list.append(df)
    multiplex_trade_df = pd.concat(multiplex_trade_dfs_list).reset_index(drop=True)
    
    #
    multiplex_lease_dfs_list = []
    for filename in multiplex_lease_files:
        df = file_reader(basedir, filename)
        multiplex_lease_dfs_list.append(df)
    multiplex_lease_df = pd.concat(multiplex_lease_dfs_list).reset_index(drop=True)
    
    #
    officetel_trade_dfs_list = []
    for filename in officetel_trade_files:
        df = file_reader(basedir, filename)
        officetel_trade_dfs_list.append(df)
    officetel_trade_df = pd.concat(officetel_trade_dfs_list).reset_index(drop=True)
    
    #
    officetel_lease_dfs_list = []
    for filename in officetel_lease_files:
        df = file_reader(basedir, filename)
        officetel_lease_dfs_list.append(df)
    officetel_lease_df = pd.concat(officetel_lease_dfs_list).reset_index(drop=True)
    
    
    return apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df

In [5]:
%%time
apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df\
= read_real_estate_prices_df()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\magics\execution.py:1321: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


Wall time: 5min 42s


In [6]:
apart_trade_df.shape

(1168192, 12)

In [7]:
apart_trade_df.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,"58,250",4,1987,언주로 3
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,"60,000",5,1987,언주로 3


In [8]:
apart_trade_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168192 entries, 0 to 1168191
Data columns (total 12 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   시군구       1168192 non-null  object 
 1   번지        1167969 non-null  object 
 2   본번        1168192 non-null  int64  
 3   부번        1168192 non-null  int64  
 4   단지명       1168192 non-null  object 
 5   전용면적(㎡)   1168192 non-null  float64
 6   계약년월      1168192 non-null  int64  
 7   계약일       1168192 non-null  int64  
 8   거래금액(만원)  1168192 non-null  object 
 9   층         1168192 non-null  int64  
 10  건축년도      1168192 non-null  int64  
 11  도로명       1168192 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 107.0+ MB


In [9]:
# '거래금액(만원)' dtype object 임. numeric으로 수정 필요.

# '번지' null 조사

In [10]:
apart_lease_df.shape

(1599426, 14)

In [11]:
apart_lease_df.head()

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,"35,000",0,7.0,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,"20,000",0,8.0,1988,언주로 103
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,"24,000",0,5.0,1988,언주로 103
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,"31,000",0,9.0,1988,언주로 103
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,"30,500",0,9.0,1988,언주로 103


In [12]:
apart_lease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599426 entries, 0 to 1599425
Data columns (total 14 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1599426 non-null  object 
 1   번지       1598203 non-null  object 
 2   본번       1599426 non-null  int64  
 3   부번       1599426 non-null  int64  
 4   단지명      1599426 non-null  object 
 5   전월세구분    1599426 non-null  object 
 6   전용면적(㎡)  1599390 non-null  float64
 7   계약년월     1599426 non-null  int64  
 8   계약일      1599426 non-null  int64  
 9   보증금(만원)  1599426 non-null  object 
 10  월세(만원)   1599426 non-null  object 
 11  층        1599390 non-null  float64
 12  건축년도     1599426 non-null  int64  
 13  도로명      1599426 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 170.8+ MB


In [13]:
# 보증금(만원) dtype object 임. numeric으로 수정 필요.
# 월세(만원) dtype object 임. numeric으로 수정 필요.
# 층 dtype float 임. int 로 수정 필요.

# 월세(만원) 0밖에 안 보임. 조사 필요.
# 번지, 전용면적,층 null 조사

In [14]:
multiplex_trade_df.shape

(662345, 13)

In [15]:
multiplex_trade_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),대지권면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201304,26,"25,000",3,2012.0,논현로6길 22-6
1,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201306,4,"25,000",3,2012.0,논현로6길 22-6
2,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201310,25,"24,500",2,2012.0,논현로6길 22-6
3,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),37.18,23.86,201302,13,"27,800",5,2012.0,개포로26길 16
4,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),35.87,23.02,201304,26,"26,500",3,2012.0,개포로26길 16


In [16]:
multiplex_trade_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662345 entries, 0 to 662344
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       662345 non-null  object 
 1   번지        662297 non-null  object 
 2   본번        662345 non-null  int64  
 3   부번        662345 non-null  int64  
 4   건물명       662342 non-null  object 
 5   전용면적(㎡)   662345 non-null  float64
 6   대지권면적(㎡)  662345 non-null  float64
 7   계약년월      662345 non-null  int64  
 8   계약일       662345 non-null  int64  
 9   거래금액(만원)  662345 non-null  object 
 10  층         662345 non-null  int64  
 11  건축년도      662317 non-null  float64
 12  도로명       662345 non-null  object 
dtypes: float64(3), int64(5), object(5)
memory usage: 65.7+ MB


In [17]:
# 거래금액(만원) dtype object임. numeric으로 수정 필요.
# 건축년도 dtype float임. int로 수정 필요.

# 번지, 건물명, 건축년도 null 조사

In [18]:
multiplex_lease_df.shape

(820486, 14)

In [19]:
multiplex_lease_df.head()

,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1252-8,1252,8,(1252-8),전세,65.92,201103,26,"20,000",0,4.0,2006.0,개포로21길 21
1,서울특별시 강남구 개포동,1252-8,1252,8,(1252-8),월세,65.92,201110,13,"15,000",50,5.0,2006.0,개포로21길 21
2,서울특별시 강남구 개포동,1265-10,1265,10,(1265-10),전세,81.12,201102,15,"23,000",0,4.0,2006.0,개포로31길 23-10
3,서울특별시 강남구 개포동,159-2,159,2,(159-2),전세,57.55,201110,11,"15,000",0,2.0,1987.0,선릉로8길 12
4,서울특별시 강남구 개포동,162-6,162,6,(162-6),전세,63.63,201103,19,"14,000",0,2.0,1987.0,선릉로10길 22-7


In [20]:
multiplex_lease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820486 entries, 0 to 820485
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      820486 non-null  object 
 1   번지       820450 non-null  object 
 2   본번       820486 non-null  int64  
 3   부번       820486 non-null  int64  
 4   건물명      820473 non-null  object 
 5   전월세구분    820486 non-null  object 
 6   전용면적(㎡)  820486 non-null  float64
 7   계약년월     820486 non-null  int64  
 8   계약일      820486 non-null  int64  
 9   보증금(만원)  820486 non-null  object 
 10  월세(만원)   820486 non-null  object 
 11  층        802481 non-null  float64
 12  건축년도     820463 non-null  float64
 13  도로명      820486 non-null  object 
dtypes: float64(3), int64(4), object(7)
memory usage: 87.6+ MB


In [21]:
# 보증금(만원) dtype object임. numeric으로 수정 필요.
# 월세(만원) dtype ojbect임. numeric으로 수정 필요.
# 층 dtype float임. int로 수정 필요.
# 건축년도 dtype float임. int로 수정 필요.

# 번지, 건물명, 층, 건축년도 null 조사

In [22]:
officetel_trade_df.shape

(171647, 12)

In [23]:
officetel_trade_df.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,157-9,157,9,\t(157-9)\t,29.25,201310,15,"16,750",3,2011.0,선릉로4길 19
1,서울특별시 강남구 개포동,13-3,13,3,대청타워,35.22,201301,10,"15,600",7,1997.0,개포로 623
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,201301,10,"15,500",6,1997.0,개포로 623
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,31.91,201301,24,"15,600",23,1997.0,개포로 623
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,43.24,201301,29,"22,000",10,1997.0,개포로 623


In [24]:
officetel_trade_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171647 entries, 0 to 171646
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       171647 non-null  object 
 1   번지        171471 non-null  object 
 2   본번        171647 non-null  int64  
 3   부번        171647 non-null  int64  
 4   단지명       171647 non-null  object 
 5   전용면적(㎡)   171647 non-null  float64
 6   계약년월      171647 non-null  int64  
 7   계약일       171647 non-null  int64  
 8   거래금액(만원)  171647 non-null  object 
 9   층         171647 non-null  int64  
 10  건축년도      170497 non-null  float64
 11  도로명       171647 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 15.7+ MB


In [25]:
# 거래금액(만원) dtype object임. numeric으로 수정 필요.
# 건축년도 dtype float임. int로 수정 필요.

# 번지, 건축년도 null 조사

In [26]:
officetel_lease_df.shape

(280365, 14)

In [27]:
officetel_lease_df.head()

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,157-9,157,9,\t(157-9)\t,전세,29.25,201112,8,"2,000",0,3,2011.0,선릉로4길 19
1,서울특별시 강남구 개포동,1246-3,1246,3,골든빌오피스텔(1246-3),월세,23.48,201112,3,200,85,4,2003.0,개포로 213
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201103,12,"10,100",0,26,1997.0,개포로 623
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201105,23,"10,000",0,18,1997.0,개포로 623
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201108,27,"10,000",0,20,1997.0,개포로 623


In [28]:
officetel_lease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280365 entries, 0 to 280364
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      280365 non-null  object 
 1   번지       279536 non-null  object 
 2   본번       280365 non-null  int64  
 3   부번       280365 non-null  int64  
 4   단지명      280365 non-null  object 
 5   전월세구분    280365 non-null  object 
 6   전용면적(㎡)  280365 non-null  float64
 7   계약년월     280365 non-null  int64  
 8   계약일      280365 non-null  int64  
 9   보증금(만원)  280365 non-null  object 
 10  월세(만원)   280365 non-null  object 
 11  층        280365 non-null  int64  
 12  건축년도     272014 non-null  float64
 13  도로명      280365 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 29.9+ MB


In [29]:
# 보증금 dtype object임. numeric으로 수정 필요.
# 월세 dtype object임. numeric으로 수정 필요.
# 건축년도 dtype float임. int로 수정 필요.

# 번지, 건축년도 null 조사.

# EDA & Preprocessing 1

### 위의 Read Data에서 .head로 훑어본 결과 월세의 대부분이 0이었음. 이에 따라 조사함.

In [30]:
apart_lease_df.head(50)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,"35,000",0,7.0,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,"20,000",0,8.0,1988,언주로 103
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,"24,000",0,5.0,1988,언주로 103
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,"31,000",0,9.0,1988,언주로 103
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,"30,500",0,9.0,1988,언주로 103
5,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201103,10,"32,000",0,4.0,1988,언주로 103
6,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201107,10,"30,000",0,1.0,1988,언주로 103
7,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201107,13,"28,000",0,9.0,1988,언주로 103
8,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201107,21,"37,000",0,1.0,1988,언주로 103
9,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201107,23,"31,000",0,6.0,1988,언주로 103


#### 조사 결과: 전월세구분 콜럼 값이 전세인 경우 0인 것 같음.

### apart_trade_df 의 '번지' 콜럼 null 조사

In [31]:
def show_bunji_null(df):
    df = df.copy()
    
    bunji_null_df = df[df['번지'].isna()]
    print(bunji_null_df.shape)
    display(bunji_null_df.head())

In [32]:
show_bunji_null(apart_trade_df)

(223, 12)


,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
73590,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,101.51,201403,18,"65,532",6,2014,밤고개로21길 25
73591,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,84.89,201404,28,"77,100",7,2014,밤고개로21길 25
73592,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,101.51,201405,12,"63,769",1,2014,밤고개로21길 25
73593,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,101.51,201405,12,"64,525",2,2014,밤고개로21길 25
73594,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,101.51,201405,12,"63,769",2,2014,밤고개로21길 25


#### 번지 null 조사 결과: 인터넷 검색 결과 단순 누락인 것으로 보임. 도로명 주소만 있으면 되니 굳이 채워 넣지 않아도 됨.

### apart_lease_df 번지, 전용면적, 층 null 조사

In [33]:
show_bunji_null(apart_lease_df)

(1223, 14)


,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
469783,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,전세,59.97,201401,10,"22,400",0,3.0,2014,밤고개로21길 25
469784,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,전세,84.97,201401,24,"27,600",0,4.0,2014,밤고개로21길 25
469785,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,전세,59.92,201401,24,"22,400",0,10.0,2014,밤고개로21길 25
469786,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,전세,84.85,201401,24,"27,600",0,6.0,2014,밤고개로21길 25
469787,서울특별시 강남구 자곡동,NaN,0,0,래미안포레,전세,59.97,201401,24,"22,400",0,4.0,2014,밤고개로21길 25


#### 번지 조사 결과: 번지 null 이유는 위 apart_trade_df 에서의 이유와 같아 보임.

In [34]:
area_null_df = apart_lease_df[apart_lease_df['전용면적(㎡)'].isna()]
area_null_df.shape

(36, 14)

In [35]:
area_null_df

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
87713,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,NaN,201109,30,"23,000",0,NaN,2012,연희로37길 25-14
144951,서울특별시 종로구 명륜3가,1-102,1,102,(1-102),전세,NaN,201106,9,"2,500",0,NaN,1997,명륜2길 9-14
170943,서울특별시 강동구 성내동,199-18,199,18,아이퍼스트빌,전세,NaN,201203,24,"11,000",0,NaN,2012,천호대로174길 16
170944,서울특별시 강동구 성내동,199-18,199,18,아이퍼스트빌,전세,NaN,201204,12,"11,000",0,NaN,2012,천호대로174길 16
199618,서울특별시 금천구 시흥동,791-40,791,40,SM엘루이,전세,NaN,201211,15,"24,000",0,NaN,2012,시흥대로37길 20
232323,서울특별시 마포구 서교동,490,490,0,메세나폴리스,전세,NaN,201206,12,"75,000",0,NaN,2012,양화로 45
232326,서울특별시 마포구 서교동,490,490,0,메세나폴리스,전세,NaN,201207,9,"70,000",0,NaN,2012,양화로 45
232329,서울특별시 마포구 서교동,490,490,0,메세나폴리스,전세,NaN,201208,6,"70,000",0,NaN,2012,양화로 45
232333,서울특별시 마포구 서교동,490,490,0,메세나폴리스,전세,NaN,201208,17,"70,000",0,NaN,2012,양화로 45
283569,서울특별시 영등포구 당산동,121-204,121,204,리첸트,전세,NaN,201209,13,"8,500",0,NaN,2012,버드나루로 91


In [36]:
# 특이사항: 전용면적이 null 인 row들은 전부 층도 null임.

In [37]:
# 조사: 층이 null인 row들은 전부 전용면적도 null인가?
floor_null_df = apart_lease_df[apart_lease_df['층'].isna()]
floor_null_df.shape

(36, 14)

In [38]:
floor_null_df['전용면적(㎡)'].isna().sum() == floor_null_df.shape[0]

True

In [39]:
# 전용면적 null의 맨 첫번째 row 의 도로명 조사

inv_df = apart_lease_df[apart_lease_df['도로명'] == '연희로37길 25-14']
inv_df.shape

(9, 14)

In [40]:
inv_df

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
87713,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,NaN,201109,30,"23,000",0,NaN,2012,연희로37길 25-14
87714,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,84.90,201112,7,"16,000",0,3.0,2012,연희로37길 25-14
390592,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,84.72,201308,30,"21,000",0,5.0,2012,연희로37길 25-14
548560,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,84.90,201406,5,"20,000",0,3.0,2012,연희로37길 25-14
710795,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,104.18,201501,4,"23,000",0,2.0,2012,연희로37길 25-14
866832,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,84.72,201610,25,"25,000",0,5.0,2012,연희로37길 25-14
1022079,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,84.90,201705,20,"23,000",0,3.0,2012,연희로37길 25-14
1184096,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,월세,84.90,201807,16,"3,000",30,4.0,2012,연희로37길 25-14
1184097,서울특별시 서대문구 홍은동,265-350,265,350,홍은드림아파트,전세,84.72,201809,30,"26,500",0,5.0,2012,연희로37길 25-14


#### 전용면적, 층 조사 결과: 전용면적과 층은 중요한 feature 들이고, nan 값을 유추할 수 없고, 해당되는 row 숫자도 적으므로 모두 제거함

In [41]:
apart_lease_df.drop(apart_lease_df[apart_lease_df['전용면적(㎡)'].isna()].index, inplace=True)
apart_lease_df.shape

(1599390, 14)

In [42]:
apart_lease_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1599390 entries, 0 to 1599425
Data columns (total 14 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1599390 non-null  object 
 1   번지       1598167 non-null  object 
 2   본번       1599390 non-null  int64  
 3   부번       1599390 non-null  int64  
 4   단지명      1599390 non-null  object 
 5   전월세구분    1599390 non-null  object 
 6   전용면적(㎡)  1599390 non-null  float64
 7   계약년월     1599390 non-null  int64  
 8   계약일      1599390 non-null  int64  
 9   보증금(만원)  1599390 non-null  object 
 10  월세(만원)   1599390 non-null  object 
 11  층        1599390 non-null  float64
 12  건축년도     1599390 non-null  int64  
 13  도로명      1599390 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 183.0+ MB


In [43]:
# 전용면적과 층 null들을 median 값으로 교체 시 사용

def area_floor_fillna_median(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'].str.split(' ').apply(lambda x: x[0]) + ' ' + df['시군구'].str.split(' ').apply(lambda x: x[1])\
                           + ' ' + df['도로명'] + ' ' + df['단지명']
    
    area_null_df = df[df['전용면적(㎡)'].isna()]
    
    unique_addresses = area_null_df['temp_full_addr'].unique()
    
    for address in unique_addresses:
        addr_df = df[df['temp_full_addr'] == address]
        
        area_median = addr_df['전용면적(㎡)'].median()
        
        df.loc[addr_df[addr_df['전용면적(㎡)'].isna()].index, '전용면적(㎡)'] = area_median
        
    df = df.drop(df[df['전용면적(㎡)'].isna()].index)
        
        
    floor_null_df = df[df['층'].isna()]
    
    unique_addresses = floor_null_df['temp_full_addr'].unique()
    
    for address in unique_addresses:
        addr_df = df[df['temp_full_addr'] == address]
        
        floor_median = addr_df['층'].median()
        
        df.loc[addr_df[addr_df['층'].isna()].index, '층'] = floor_median
        
    df = df.drop(df[df['층'].isna()].index)
        
    return df

### multiplex_trade_df 번지, 건물명, 건축년도 null 조사

In [44]:
print(multiplex_trade_df.shape)
multiplex_trade_df.head()

(662345, 13)


,시군구,번지,본번,부번,건물명,전용면적(㎡),대지권면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201304,26,"25,000",3,2012.0,논현로6길 22-6
1,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201306,4,"25,000",3,2012.0,논현로6길 22-6
2,서울특별시 강남구 개포동,1172-2,1172,2,(1172-2),42.15,24.67,201310,25,"24,500",2,2012.0,논현로6길 22-6
3,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),37.18,23.86,201302,13,"27,800",5,2012.0,개포로26길 16
4,서울특별시 강남구 개포동,1216-4,1216,4,(1216-4),35.87,23.02,201304,26,"26,500",3,2012.0,개포로26길 16


In [45]:
show_bunji_null(multiplex_trade_df)

(48, 13)


,시군구,번지,본번,부번,건물명,전용면적(㎡),대지권면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
11567,서울특별시 노원구 상계동,NaN,0,0,수암빌딩,115.91,36.50,201303,2,"17,000",4,1995.0,덕릉로 833
41492,서울특별시 노원구 상계동,NaN,0,0,그린하이츠,62.88,26.38,201410,6,"17,000",1,2002.0,상계로37가길 18-17
41602,서울특별시 노원구 상계동,NaN,0,0,신아빌리지2차가동,49.89,36.61,201402,14,"12,000",2,1990.0,덕릉로124가길 13-12
41603,서울특별시 노원구 상계동,NaN,0,0,신아빌리지2차가동,49.89,36.61,201406,28,"12,800",1,1990.0,덕릉로124가길 13-12
83772,서울특별시 노원구 상계동,NaN,0,0,그린하이츠,60.28,25.28,201501,6,"17,000",3,2002.0,상계로37가길 18-17


#### 번지 null 조사 결과: 위 다른 df들의 번지 null 이유와 같음

In [46]:
bdname_null_df = multiplex_trade_df[multiplex_trade_df['건물명'].isna()]
bdname_null_df.shape

(3, 13)

In [47]:
bdname_null_df.head()

,시군구,번지,본번,부번,건물명,전용면적(㎡),대지권면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
434958,서울특별시 강동구 명일동,121,121,0,NaN,46.65,33.37,200710,22,"31,500",-1,1986.0,동남로65길 16-18
434959,서울특별시 강동구 명일동,121,121,0,NaN,54.54,38.63,200711,21,"32,300",-1,1986.0,동남로65길 16-18
491756,서울특별시 강동구 명일동,121,121,0,NaN,46.65,33.37,200801,22,"33,600",-1,1986.0,동남로65길 16-18


#### 건물명 null 조사 결과: 이유 유추 불가, 중요하지 않은 feature 이므로, 제거하지 않기로 함.

In [48]:
byear_null_df = multiplex_trade_df[multiplex_trade_df['건축년도'].isna()]
byear_null_df.shape

(28, 13)

In [49]:
byear_null_df

,시군구,번지,본번,부번,건물명,전용면적(㎡),대지권면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
18385,서울특별시 성북구 길음동,1083,1083,0,대원연립,46.84,16.43,201301,11,"19,800",3,NaN,삼양로2길 19-5
50711,서울특별시 성북구 길음동,1083,1083,0,대원연립,59.50,20.86,201402,15,"17,200",4,NaN,삼양로2길 19-5
153477,서울특별시 성북구 길음동,1083,1083,0,대원연립,59.50,19.83,201602,21,"23,500",4,NaN,삼양로2길 19-5
153478,서울특별시 성북구 길음동,1083,1083,0,대원연립,60.10,20.83,201603,8,"22,000",2,NaN,삼양로2길 19-5
153479,서울특별시 성북구 길음동,1083,1083,0,대원연립,46.84,16.53,201605,14,"18,200",3,NaN,삼양로2길 19-5
153480,서울특별시 성북구 길음동,1083,1083,0,대원연립,46.28,16.23,201607,20,"18,700",4,NaN,삼양로2길 19-5
207079,서울특별시 성북구 길음동,1083,1083,0,대원연립,46.38,16.26,201706,16,"24,000",2,NaN,삼양로2길 19-5
207080,서울특별시 성북구 길음동,1083,1083,0,대원연립,52.89,18.55,201712,8,"24,500",4,NaN,삼양로2길 19-5
207081,서울특별시 성북구 길음동,1083,1083,0,대원연립,59.50,20.86,201712,29,"26,000",4,NaN,삼양로2길 19-5
255347,서울특별시 성북구 길음동,1083,1083,0,대원연립,39.70,33.06,201801,17,"25,800",-1,NaN,삼양로2길 19-5


#### 건축년도 null 조사 결과: 이유 유추 불가, 중요한 feature 이므로 제거하지 않기로 결정. 인터넷 검색을 통해 fill 하기로 결정 (검색 결과: 1971년).

In [50]:
# multiplex_trade_df 건축년도 null fill

multiplex_trade_df.loc[byear_null_df.index, '건축년도'] = 1971

### multiplex_lease_df 번지, 건물명, 층, 건축년도 null 조사

In [51]:
print(multiplex_lease_df.shape)
multiplex_lease_df.head()

(820486, 14)


,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1252-8,1252,8,(1252-8),전세,65.92,201103,26,"20,000",0,4.0,2006.0,개포로21길 21
1,서울특별시 강남구 개포동,1252-8,1252,8,(1252-8),월세,65.92,201110,13,"15,000",50,5.0,2006.0,개포로21길 21
2,서울특별시 강남구 개포동,1265-10,1265,10,(1265-10),전세,81.12,201102,15,"23,000",0,4.0,2006.0,개포로31길 23-10
3,서울특별시 강남구 개포동,159-2,159,2,(159-2),전세,57.55,201110,11,"15,000",0,2.0,1987.0,선릉로8길 12
4,서울특별시 강남구 개포동,162-6,162,6,(162-6),전세,63.63,201103,19,"14,000",0,2.0,1987.0,선릉로10길 22-7


In [52]:
show_bunji_null(multiplex_lease_df)

(36, 14)


,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
24016,서울특별시 노원구 상계동,NaN,0,0,금정빌라,전세,64.37,201112,16,"10,000",0,3.0,1995.0,상계로33길 6-5
24177,서울특별시 노원구 상계동,NaN,0,0,상진아트빌,전세,60.39,201110,10,"7,000",0,1.0,2002.0,상계로33길 52-10
24210,서울특별시 노원구 상계동,NaN,0,0,수암빌딩,전세,62.13,201106,27,"5,500",0,4.0,1995.0,덕릉로 833
24220,서울특별시 노원구 상계동,NaN,0,0,신아빌리지2차가동,전세,50.49,201101,24,"5,000",0,NaN,1990.0,덕릉로124가길 13-12
24221,서울특별시 노원구 상계동,NaN,0,0,신아빌리지2차가동,전세,36.96,201109,23,"5,500",0,NaN,1990.0,덕릉로124가길 13-12


#### 번지 null 조사 결과: 위 다른 케이스들의 이유와 같음.

In [53]:
bdname_null_df = multiplex_lease_df[multiplex_lease_df['건물명'].isna()]
bdname_null_df.shape

(13, 14)

In [54]:
bdname_null_df

,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
4515,서울특별시 강동구 명일동,121,121,0,NaN,전세,54.54,201102,10,"10,000",0,NaN,1986.0,동남로65길 16-18
4516,서울특별시 강동구 명일동,121,121,0,NaN,전세,42.90,201102,23,"10,000",0,NaN,1986.0,동남로65길 16-18
4517,서울특별시 강동구 명일동,121,121,0,NaN,전세,46.65,201103,8,"10,000",0,2.0,1986.0,동남로65길 16-18
4518,서울특별시 강동구 명일동,121,121,0,NaN,전세,46.65,201110,1,"10,000",0,NaN,1986.0,동남로65길 16-18
75887,서울특별시 강동구 명일동,121,121,0,NaN,전세,46.65,201205,1,"11,000",0,1.0,1986.0,동남로65길 16-18
148565,서울특별시 강동구 명일동,121,121,0,NaN,전세,54.54,201303,4,"10,000",0,1.0,1986.0,동남로65길 16-18
148566,서울특별시 강동구 명일동,121,121,0,NaN,전세,54.54,201303,6,"12,000",0,NaN,1986.0,동남로65길 16-18
223508,서울특별시 강동구 명일동,121,121,0,NaN,전세,46.65,201403,15,"11,000",0,1.0,1986.0,동남로65길 16-18
300870,서울특별시 강동구 명일동,121,121,0,NaN,전세,46.65,201504,14,"13,500",0,2.0,1986.0,동남로65길 16-18
300871,서울특별시 강동구 명일동,121,121,0,NaN,월세,54.54,201508,6,"10,000",35,2.0,1986.0,동남로65길 16-18


In [55]:
# 특이사항: 층 null, 모두 같은 주소

#### 건물명 null 조사 결과: 중요하지 않으므로 내버려 둠.

In [56]:
# 층 null 조사
floor_null_df = multiplex_lease_df[multiplex_lease_df['층'].isna()]
floor_null_df.shape

(18005, 14)

In [57]:
floor_null_df.head()

,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
12,서울특별시 강남구 개포동,171-13,171,13,(171-13),월세,49.50,201102,15,"4,000",10,NaN,1988.0,선릉로14길 11
52,서울특별시 강남구 개포동,1219-8,1219,8,금호빌,전세,42.00,201101,20,"12,000",0,NaN,1997.0,개포로20길 43
110,서울특별시 강남구 개포동,157,157,0,대영빌딩(157-0),전세,76.00,201104,28,"19,000",0,NaN,2003.0,선릉로6길 20
120,서울특별시 강남구 개포동,1167-6,1167,6,동서울 주택,월세,42.82,201103,11,"4,000",10,NaN,1995.0,논현로2길 35
156,서울특별시 강남구 개포동,1181-7,1181,7,백촌빌라,월세,37.17,201109,2,"2,000",40,NaN,1989.0,논현로8길 62-5


In [58]:
multiplex_lease_df.shape

(820486, 14)

In [59]:
floor_null_df.shape[0] / multiplex_lease_df.shape[0]

0.021944311054667598

#### 층 null 조사 결과: 얼마 되지 않으므로 제거하기로 함

In [60]:
# 층 null 제거

multiplex_lease_df.drop(floor_null_df.index, inplace=True)
multiplex_lease_df.shape

(802481, 14)

In [61]:
# 건축년도 null 조사

bltyear_null_df = multiplex_lease_df[multiplex_lease_df['건축년도'].isna()]
bltyear_null_df.shape

(19, 14)

In [62]:
bltyear_null_df.head()

,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
42752,서울특별시 성북구 길음동,1083,1083,0,대원연립,월세,60.10,201102,11,"1,000",35,2.0,NaN,삼양로2길
42753,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,46.88,201110,11,"4,500",0,2.0,NaN,삼양로2길
189731,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,73.62,201305,29,"7,000",0,4.0,NaN,삼양로2길
265962,서울특별시 성북구 길음동,1083,1083,0,대원연립,월세,60.07,201404,19,"5,000",30,2.0,NaN,삼양로2길 19-5
265963,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,60.07,201412,17,"8,000",0,0.0,NaN,


## 중요 특이사항: 비어있는 도로명 조사 필요.

In [63]:
# 비어있는 도로명 조사

bltyear_null_df.loc[265963, '도로명']

' '

In [64]:
bltyear_null_df.loc[265963, '도로명'] == ' '

True

In [65]:
empty_roadname_df = multiplex_lease_df[multiplex_lease_df['도로명'] == ' ']
print(empty_roadname_df.shape)
empty_roadname_df.head()

(1658, 14)


,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
738,서울특별시 강남구 논현동,34,34,0,동방타운A동,전세,26.67,201112,24,"6,000",0,0.0,1996.0,
2394,서울특별시 강남구 삼성동,71-3,71,3,테크빌,전세,71.07,201108,23,"26,000",0,3.0,2002.0,
3378,서울특별시 강남구 역삼동,745-4,745,4,역삼빌라B(745-4),전세,34.95,201107,23,"13,000",0,1.0,1988.0,
3379,서울특별시 강남구 역삼동,745-4,745,4,역삼빌라B(745-4),월세,34.95,201109,19,500,60,0.0,1988.0,
4126,서울특별시 강동구 길동,103-3,103,3,길동주택(B동),전세,41.17,201111,4,"7,500",0,0.0,1986.0,


#### 비어있는 도로명 조사 결과: 인터넷 검색 결과, 도로명은 존재함. 단순 누락인 것으로 보임. 다른 모든 df 들도 조사 필요.

In [66]:
# df별 각 콜럼마다 얼마나 많은 빈 값(' ')이 있는지 조사. 또한, 빈 값이 ' '(스페이스 한칸)인지, 여러 칸인지 (예: '  ') 조사.

def replace_spaces_to_empty(x):    
    if type(x) == str:
        return x.replace(' ', '')
    else:
        return x

In [67]:
def report_empty_numb(df):
    df = df.copy()
    
    for col in df.columns:
        df[col] = df[col].apply(replace_spaces_to_empty)
        
    for col in df.columns:
        print(col + ': ', (df[col] == '').sum())

In [68]:
# apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df

In [69]:
print(apart_trade_df.shape)
report_empty_numb(apart_trade_df)

(1168192, 12)
시군구:  0
번지:  0
본번:  0
부번:  0
단지명:  0
전용면적(㎡):  0
계약년월:  0
계약일:  0
거래금액(만원):  0
층:  0
건축년도:  0
도로명:  2220


In [70]:
(apart_trade_df['도로명'] == ' ').sum()

2220

In [71]:
print(apart_lease_df.shape)
report_empty_numb(apart_lease_df)

(1599390, 14)
시군구:  0
번지:  0
본번:  0
부번:  0
단지명:  0
전월세구분:  0
전용면적(㎡):  0
계약년월:  0
계약일:  0
보증금(만원):  0
월세(만원):  0
층:  0
건축년도:  0
도로명:  13505


In [72]:
(apart_lease_df['도로명'] == ' ').sum()

13505

In [73]:
print(multiplex_trade_df.shape)
report_empty_numb(multiplex_trade_df)

(662345, 13)
시군구:  0
번지:  0
본번:  0
부번:  0
건물명:  0
전용면적(㎡):  0
대지권면적(㎡):  0
계약년월:  0
계약일:  0
거래금액(만원):  0
층:  0
건축년도:  0
도로명:  1269


In [74]:
(multiplex_trade_df['도로명'] == ' ').sum()

1269

In [75]:
print(multiplex_lease_df.shape)
report_empty_numb(multiplex_lease_df)

(802481, 14)
시군구:  0
번지:  0
본번:  0
부번:  0
건물명:  0
전월세구분:  0
전용면적(㎡):  0
계약년월:  0
계약일:  0
보증금(만원):  0
월세(만원):  0
층:  0
건축년도:  0
도로명:  1658


In [76]:
(multiplex_lease_df['도로명'] == ' ').sum()

1658

In [77]:
print(officetel_trade_df.shape)
report_empty_numb(officetel_trade_df)

(171647, 12)
시군구:  0
번지:  0
본번:  0
부번:  0
단지명:  0
전용면적(㎡):  0
계약년월:  0
계약일:  0
거래금액(만원):  0
층:  0
건축년도:  0
도로명:  3515


In [78]:
(officetel_trade_df['도로명'] == ' ').sum()

3515

In [79]:
print(officetel_lease_df.shape)
report_empty_numb(officetel_lease_df)

(280365, 14)
시군구:  0
번지:  0
본번:  0
부번:  0
단지명:  0
전월세구분:  0
전용면적(㎡):  0
계약년월:  0
계약일:  0
보증금(만원):  0
월세(만원):  0
층:  0
건축년도:  0
도로명:  31997


In [80]:
(officetel_lease_df['도로명'] == ' ').sum()

31997

#### 조사 결과: 모든 df들이 도로명 콜럼만 비어있는 값이 있음. 또한, 모든 비어있는 값은 ' ' (스페이스 한칸)임. 여러 칸인 경우는 없음.

#### 당장은 지번 주소만 있으면 되니 일단 빈 칸을 null로 교체. 나중에 필요하면 주소 인터넷에서 받아와서 채우기.

In [81]:
# 모든 df 도로명 빈 칸 null 로 교체

def replace_empty_to_null(df):
    df = df.copy()
    
    empty_df = df[df['도로명'] == ' ']
    
    df.loc[empty_df.index, '도로명'] = np.nan
    
    return df

In [82]:
# apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df

In [83]:
apart_trade_df = replace_empty_to_null(apart_trade_df)
apart_trade_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168192 entries, 0 to 1168191
Data columns (total 12 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   시군구       1168192 non-null  object 
 1   번지        1167969 non-null  object 
 2   본번        1168192 non-null  int64  
 3   부번        1168192 non-null  int64  
 4   단지명       1168192 non-null  object 
 5   전용면적(㎡)   1168192 non-null  float64
 6   계약년월      1168192 non-null  int64  
 7   계약일       1168192 non-null  int64  
 8   거래금액(만원)  1168192 non-null  object 
 9   층         1168192 non-null  int64  
 10  건축년도      1168192 non-null  int64  
 11  도로명       1165972 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 107.0+ MB


In [84]:
apart_trade_df['도로명'].isna().sum()

2220

In [85]:
apart_lease_df = replace_empty_to_null(apart_lease_df)
multiplex_trade_df = replace_empty_to_null(multiplex_trade_df)
multiplex_lease_df = replace_empty_to_null(multiplex_lease_df)
officetel_trade_df = replace_empty_to_null(officetel_trade_df)
officetel_lease_df = replace_empty_to_null(officetel_lease_df)

### 건축년도 null 계속해서 조사

In [86]:
print(bltyear_null_df.shape)
bltyear_null_df

(19, 14)


,시군구,번지,본번,부번,건물명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
42752,서울특별시 성북구 길음동,1083,1083,0,대원연립,월세,60.10,201102,11,"1,000",35,2.0,NaN,삼양로2길
42753,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,46.88,201110,11,"4,500",0,2.0,NaN,삼양로2길
189731,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,73.62,201305,29,"7,000",0,4.0,NaN,삼양로2길
265962,서울특별시 성북구 길음동,1083,1083,0,대원연립,월세,60.07,201404,19,"5,000",30,2.0,NaN,삼양로2길 19-5
265963,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,60.07,201412,17,"8,000",0,0.0,NaN,
343787,서울특별시 성북구 길음동,1083,1083,0,대원연립,월세,60.10,201506,10,"2,500",30,2.0,NaN,삼양로2길 19-5
423423,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,60.07,201601,11,"8,500",0,3.0,NaN,삼양로2길 15
423424,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,59.50,201605,4,"6,700",0,4.0,NaN,삼양로2길 19-5
423425,서울특별시 성북구 길음동,1083,1083,0,대원연립,전세,46.28,201608,4,"4,500",0,4.0,NaN,삼양로2길 19-5
423426,서울특별시 성북구 길음동,1083,1083,0,대원연립,월세,60.10,201609,28,"2,500",35,2.0,NaN,삼양로2길 19-5


#### multiplex_lease_df 건축년도 null 조사 결과: 간단한 인터넷 검색으로 1971년 준공인 것을 찾을 수 있었음.

In [87]:
# 건축년도 1971년 입력

multiplex_lease_df.loc[bltyear_null_df.index, '건축년도'] = 1971

### officetel_trade_df 번지, 건축년도 null 조사

In [88]:
officetel_trade_df.shape

(171647, 12)

In [89]:
# 번지 null 조사

show_bunji_null(officetel_trade_df)

(176, 12)


,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
10032,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201406,2,"12,444",8,2014.0,NaN
10033,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201408,15,"13,191",4,2014.0,NaN
10034,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201408,15,"13,191",7,2014.0,NaN
10035,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201408,19,"13,191",6,2014.0,NaN
10036,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,24.14,201411,19,"12,444",8,2014.0,NaN


#### 번지 null 조사 결과: 위 df들과 같음.

In [90]:
# 건축년도 null 조사

bltyear_null_df = officetel_trade_df[officetel_trade_df['건축년도'].isna()]
print(bltyear_null_df.shape)
bltyear_null_df.head(50)

(1150, 12)


,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
17201,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,15.91,201512,1,"10,700",5,NaN,NaN
17202,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,15.91,201512,14,"10,700",5,NaN,NaN
29886,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,15.91,201601,4,"10,700",5,NaN,NaN
29887,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,16.34,201603,28,"9,900",8,NaN,NaN
29888,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,16.34,201603,31,"9,364",6,NaN,NaN
33249,서울특별시 금천구 독산동,378-461,378,461,문성빌,59.57,201601,27,"26,200",7,NaN,독산로52길 22
33274,서울특별시 금천구 독산동,1009-75,1009,75,에이엔캐슬,48.59,201605,10,"21,900",3,NaN,시흥대로101길 8-3
33953,서울특별시 동대문구 용두동,104-3,104,3,동대문 푸르지오시티,23.47,201609,10,"13,440",3,NaN,NaN
33954,서울특별시 동대문구 용두동,104-3,104,3,동대문 푸르지오시티,23.47,201609,19,"15,570",20,NaN,NaN
33955,서울특별시 동대문구 용두동,104-3,104,3,동대문 푸르지오시티,23.47,201609,24,"14,240",5,NaN,NaN


In [91]:
# 건축년도 null인 row 중에 몇 종류의 주소가 있는지 조사

def show_how_many_addrs(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'] + df['번지'] + df['단지명']
    
    print(df['temp_full_addr'].nunique())

In [92]:
show_how_many_addrs(bltyear_null_df)

97


In [93]:
# 건축년도 null인 row의 비중이 얼마나 되는지 비교

bltyear_null_df.shape[0] / officetel_trade_df.shape[0]

0.006699796675735667

In [94]:
# 건축년도가 null 인 row 들에 있는 주소들 중에서, 같은 주소지만 건축년도 값이 있는 주소가 있는지 조사

def addr_check(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'] + df['번지'] + df['단지명']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    unique_addrs = bltyear_null_df['temp_full_addr'].unique()
    
    count = 0
    for addr in unique_addrs:
        addr_df = df[df['temp_full_addr'] == addr]
        if addr_df['건축년도'].isna().sum() != addr_df.shape[0]:
            count += 1
            
    if count != 0:
        print('건축년도 콜럼에 null과 제대로 된 값이 모두 포함된 주소가 있음')

In [95]:
addr_check(officetel_trade_df)

건축년도 콜럼에 null과 제대로 된 값이 모두 포함된 주소가 있음


#### 건축년도 null 조사 결과: 건축년도 null 인 row 들에 있는 주소들 중에서, 그 주소들로 검색해봤을 때, 같은 주소에 건축년도 값이 있는 것을 확인함.

#### 따라서, 위에 해당하는 주소는 null 을 채워놓고, 해당하지 않고 null 밖에 없는 주소들은 제거함.

In [96]:
# 건축년도가 null 인 row 들에 있는 주소들 중에서, 같은 주소지만 건축년도 값이 있는 주소는 null 을 채워넣고, 없는 주소는 df 에서 제거.
# 추가적으로, 한 주소지, 단지이름에 건축년도가 두 가지 이상 있는지도 조사함.

def bltyear_fill_and_remove_null(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'] + df['번지'] + df['단지명']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    unique_addrs = bltyear_null_df['temp_full_addr'].unique()
    
    concat_list = []
    for addr in unique_addrs:
        addr_df = df[df['temp_full_addr'] == addr]
        if addr_df['건축년도'].isna().sum() != addr_df.shape[0]:
            if addr_df['건축년도'].std() != 0:
                print('건축년도가 두 가지 이상인 주소가 있음.')
            
            addr_df['건축년도'].fillna(addr_df['건축년도'].mean(), inplace=True)
            
            concat_list.append(addr_df)
    
    df.drop(columns=['temp_full_addr'], inplace=True)
    
    if len(concat_list) > 0:
        concat_df = pd.concat(concat_list)
    else:
        return df
    
    #return concat_df
    
    df.loc[concat_df.index, '건축년도'] = concat_df['건축년도']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    df.drop(bltyear_null_df.index, inplace=True)
        
    return df.reset_index(drop=True)

In [97]:
officetel_trade_df = bltyear_fill_and_remove_null(officetel_trade_df)
print(officetel_trade_df.shape)
display(officetel_trade_df.head())
officetel_trade_df.info()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


(170508, 12)


,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,157-9,157,9,\t(157-9)\t,29.25,201310,15,"16,750",3,2011.0,선릉로4길 19
1,서울특별시 강남구 개포동,13-3,13,3,대청타워,35.22,201301,10,"15,600",7,1997.0,개포로 623
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,201301,10,"15,500",6,1997.0,개포로 623
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,31.91,201301,24,"15,600",23,1997.0,개포로 623
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,43.24,201301,29,"22,000",10,1997.0,개포로 623


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170508 entries, 0 to 170507
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       170508 non-null  object 
 1   번지        170342 non-null  object 
 2   본번        170508 non-null  int64  
 3   부번        170508 non-null  int64  
 4   단지명       170508 non-null  object 
 5   전용면적(㎡)   170508 non-null  float64
 6   계약년월      170508 non-null  int64  
 7   계약일       170508 non-null  int64  
 8   거래금액(만원)  170508 non-null  object 
 9   층         170508 non-null  int64  
 10  건축년도      170508 non-null  float64
 11  도로명       167713 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 15.6+ MB


### officetel_lease_df 번지, 건축년도 null 조사

In [98]:
# 번지 null 조사

show_bunji_null(officetel_lease_df)

(829, 14)


,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
47893,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,전세,24.14,201406,24,"9,500",0,4,2014.0,NaN
47894,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,전세,24.14,201408,28,"10,000",0,6,2014.0,NaN
47895,서울특별시 구로구 천왕동,NaN,0,0,네이처힐,전세,24.14,201412,1,"10,000",0,8,2014.0,NaN
47896,서울특별시 구로구 천왕동,NaN,0,0,랑데르,전세,41.38,201409,22,"15,000",0,6,2014.0,경인로 100
47897,서울특별시 구로구 천왕동,NaN,0,0,랑데르,전세,41.38,201410,6,"14,500",0,9,2014.0,경인로 100


#### 번지 null 조사 결과: 위 df들과 이유 같음.

In [99]:
# 건축년도 null 조사

def show_bltyear_null(df):
    df = df.copy()
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    print(bltyear_null_df.shape)
    display(bltyear_null_df.head(50))

In [100]:
show_bltyear_null(officetel_lease_df)

(8351, 14)


,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
56282,서울특별시 영등포구 문래동3가,55-4,55,4,문래동메가트리움,전세,99.44,201409,19,"37,000",0,10,NaN,문래로26길 6
61776,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,전세,16.34,201511,12,"8,000",0,7,NaN,NaN
61777,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,월세,15.91,201511,23,"4,000",26,8,NaN,NaN
61778,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,월세,20.55,201511,25,"2,000",35,8,NaN,NaN
61779,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,월세,15.91,201511,27,500,40,7,NaN,NaN
61780,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,월세,20.55,201511,28,"3,000",30,7,NaN,NaN
61781,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,월세,15.91,201511,28,500,39,8,NaN,NaN
61782,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,월세,15.91,201512,12,"1,500",33,7,NaN,NaN
61783,서울특별시 강동구 강일동,677-3,677,3,트리피움오피스텔,월세,15.91,201512,15,500,40,7,NaN,NaN
65764,서울특별시 강서구 화곡동,29-42,29,42,루체쓰리,월세,18.20,201512,30,500,40,2,NaN,까치산로15길 3


In [101]:
# 건축년도가 null 인 row 들에 있는 주소들 중에서, 같은 주소지만 건축년도 값이 있는 주소가 있는지 조사

addr_check(officetel_lease_df)

건축년도 콜럼에 null과 제대로 된 값이 모두 포함된 주소가 있음


In [102]:
# 건축년도가 null 인 row 들에 있는 주소들 중에서, 같은 주소지만 건축년도 값이 있는 주소는 null 을 채워넣고, 없는 주소는 df 에서 제거.
# 추가적으로, 한 주소지, 단지이름에 건축년도가 두 가지 이상 있는지도 조사함.

officetel_lease_df = bltyear_fill_and_remove_null(officetel_lease_df)
print(officetel_lease_df.shape)
display(officetel_lease_df.head())
officetel_lease_df.info()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


(272086, 14)


,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,157-9,157,9,\t(157-9)\t,전세,29.25,201112,8,"2,000",0,3,2011.0,선릉로4길 19
1,서울특별시 강남구 개포동,1246-3,1246,3,골든빌오피스텔(1246-3),월세,23.48,201112,3,200,85,4,2003.0,개포로 213
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201103,12,"10,100",0,26,1997.0,개포로 623
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201105,23,"10,000",0,18,1997.0,개포로 623
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201108,27,"10,000",0,20,1997.0,개포로 623


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272086 entries, 0 to 272085
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      272086 non-null  object 
 1   번지       271276 non-null  object 
 2   본번       272086 non-null  int64  
 3   부번       272086 non-null  int64  
 4   단지명      272086 non-null  object 
 5   전월세구분    272086 non-null  object 
 6   전용면적(㎡)  272086 non-null  float64
 7   계약년월     272086 non-null  int64  
 8   계약일      272086 non-null  int64  
 9   보증금(만원)  272086 non-null  object 
 10  월세(만원)   272086 non-null  object 
 11  층        272086 non-null  int64  
 12  건축년도     272086 non-null  float64
 13  도로명      245491 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 29.1+ MB


## 메모리 사용량을 줄이기 위한 콜럼들 dtype 변경.

In [103]:
def trade_dfs_change_dtype(df):
    df = df.copy()
    
    df['거래금액(만원)'] = df['거래금액(만원)'].apply(str)
    df['거래금액(만원)'] = pd.to_numeric(df['거래금액(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [104]:
apart_trade_df = trade_dfs_change_dtype(apart_trade_df)
print(apart_trade_df.shape)
display(apart_trade_df.head())
apart_trade_df.info()

(1168192, 12)


,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1168192 entries, 0 to 1168191
Data columns (total 12 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   시군구       1168192 non-null  object 
 1   번지        1167969 non-null  object 
 2   본번        1168192 non-null  int64  
 3   부번        1168192 non-null  int64  
 4   단지명       1168192 non-null  object 
 5   전용면적(㎡)   1168192 non-null  float64
 6   계약년월      1168192 non-null  int64  
 7   계약일       1168192 non-null  int64  
 8   거래금액(만원)  1168192 non-null  int64  
 9   층         1168192 non-null  int32  
 10  건축년도      1168192 non-null  int32  
 11  도로명       1165972 non-null  object 
dtypes: float64(1), int32(2), int64(5), object(4)
memory usage: 98.0+ MB


In [105]:
multiplex_trade_df = trade_dfs_change_dtype(multiplex_trade_df)
officetel_trade_df = trade_dfs_change_dtype(officetel_trade_df)

display(multiplex_trade_df.info())
display(officetel_trade_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662345 entries, 0 to 662344
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       662345 non-null  object 
 1   번지        662297 non-null  object 
 2   본번        662345 non-null  int64  
 3   부번        662345 non-null  int64  
 4   건물명       662342 non-null  object 
 5   전용면적(㎡)   662345 non-null  float64
 6   대지권면적(㎡)  662345 non-null  float64
 7   계약년월      662345 non-null  int64  
 8   계약일       662345 non-null  int64  
 9   거래금액(만원)  662345 non-null  int64  
 10  층         662345 non-null  int32  
 11  건축년도      662345 non-null  int32  
 12  도로명       661076 non-null  object 
dtypes: float64(2), int32(2), int64(5), object(4)
memory usage: 60.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170508 entries, 0 to 170507
Data columns (total 12 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       170508 non-null  object 
 1   번지        170342 non-null  object 
 2   본번        170508 non-null  int64  
 3   부번        170508 non-null  int64  
 4   단지명       170508 non-null  object 
 5   전용면적(㎡)   170508 non-null  float64
 6   계약년월      170508 non-null  int64  
 7   계약일       170508 non-null  int64  
 8   거래금액(만원)  170508 non-null  int64  
 9   층         170508 non-null  int32  
 10  건축년도      170508 non-null  int32  
 11  도로명       167713 non-null  object 
dtypes: float64(1), int32(2), int64(5), object(4)
memory usage: 14.3+ MB


None

In [106]:
def lease_dfs_change_dtype(df):
    df = df.copy()
    
    df['보증금(만원)'] = df['보증금(만원)'].apply(str)
    df['월세(만원)'] = df['월세(만원)'].apply(str)
    df['보증금(만원)'] = pd.to_numeric(df['보증금(만원)'].str.replace(',', ''))
    df['월세(만원)'] = pd.to_numeric(df['월세(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [107]:
apart_lease_df = lease_dfs_change_dtype(apart_lease_df)
print(apart_lease_df.shape)
display(apart_lease_df.head())
apart_lease_df.info()

(1599390, 14)


,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8,1988,언주로 103
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5,1988,언주로 103
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9,1988,언주로 103
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9,1988,언주로 103


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1599390 entries, 0 to 1599425
Data columns (total 14 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   시군구      1599390 non-null  object 
 1   번지       1598167 non-null  object 
 2   본번       1599390 non-null  int64  
 3   부번       1599390 non-null  int64  
 4   단지명      1599390 non-null  object 
 5   전월세구분    1599390 non-null  object 
 6   전용면적(㎡)  1599390 non-null  float64
 7   계약년월     1599390 non-null  int64  
 8   계약일      1599390 non-null  int64  
 9   보증금(만원)  1599390 non-null  int64  
 10  월세(만원)   1599390 non-null  int64  
 11  층        1599390 non-null  int32  
 12  건축년도     1599390 non-null  int32  
 13  도로명      1585885 non-null  object 
dtypes: float64(1), int32(2), int64(6), object(5)
memory usage: 210.8+ MB


In [108]:
multiplex_lease_df = lease_dfs_change_dtype(multiplex_lease_df)
officetel_lease_df = lease_dfs_change_dtype(officetel_lease_df)

display(multiplex_lease_df.info())
officetel_lease_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 802481 entries, 0 to 820485
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      802481 non-null  object 
 1   번지       802457 non-null  object 
 2   본번       802481 non-null  int64  
 3   부번       802481 non-null  int64  
 4   건물명      802472 non-null  object 
 5   전월세구분    802481 non-null  object 
 6   전용면적(㎡)  802481 non-null  float64
 7   계약년월     802481 non-null  int64  
 8   계약일      802481 non-null  int64  
 9   보증금(만원)  802481 non-null  int64  
 10  월세(만원)   802481 non-null  int64  
 11  층        802481 non-null  int32  
 12  건축년도     802481 non-null  int32  
 13  도로명      800823 non-null  object 
dtypes: float64(1), int32(2), int64(6), object(5)
memory usage: 105.7+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272086 entries, 0 to 272085
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      272086 non-null  object 
 1   번지       271276 non-null  object 
 2   본번       272086 non-null  int64  
 3   부번       272086 non-null  int64  
 4   단지명      272086 non-null  object 
 5   전월세구분    272086 non-null  object 
 6   전용면적(㎡)  272086 non-null  float64
 7   계약년월     272086 non-null  int64  
 8   계약일      272086 non-null  int64  
 9   보증금(만원)  272086 non-null  int64  
 10  월세(만원)   272086 non-null  int64  
 11  층        272086 non-null  int32  
 12  건축년도     272086 non-null  int32  
 13  도로명      245491 non-null  object 
dtypes: float64(1), int32(2), int64(6), object(5)
memory usage: 27.0+ MB
